In [ ]:
import os
import time
import numpy as np
from keras import backend as K
import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(100)
from fit_on_batches import *

In [ ]:
imgPath = 'DATA/Stored/train/'
maskPath = 'DATA/Stored/train_masks/'
#DIM = (1280, 1920, 3)
DIM = (160, 240, 3)
print('Total {} samples'.format(len([i for i in os.listdir(imgPath)])))

In [ ]:
# Define IoU metric and loss
from keras.losses import binary_crossentropy

def dice_coef(y_true, y_pred, smooth=1.):
    y_true_f = K.batch_flatten(y_true)
    y_pred_f = K.batch_flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

def dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
# Build U-Net model

def conv_block(m, dim, acti, bn, res, do=0):
    n = Conv2D(dim, 3, activation=acti, padding='same')(m)
    n = BatchNormalization()(n) if bn else n
    n = Dropout(do)(n) if do else n
    n = Conv2D(dim, 3, activation=acti, padding='same')(n)
    n = BatchNormalization()(n) if bn else n
    return Concatenate()([m, n]) if res else n

def level_block(m, dim, depth, inc, acti, do, bn, mp, up, res):
    if depth > 0:
        n = conv_block(m, dim, acti, bn, res)
        m = MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
        m = level_block(m, int(inc*dim), depth-1, inc, acti, do, bn, mp, up, res)
        if up:
            m = UpSampling2D()(m)
            m = Conv2D(dim, 2, activation=acti, padding='same')(m)
        else:
            m = Conv2DTranspose(dim, 3, strides=2, activation=acti, padding='same')(m)
        n = Concatenate()([n, m])
        m = conv_block(n, dim, acti, bn, res)
    else:
        m = conv_block(m, dim, acti, bn, res, do)
    return m

def UNet(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu', dropout=0.5, batchnorm=False, maxpool=True, upconv=True, residual=False):
    
    i = Input(shape=img_shape)
    o = level_block(i, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
    o = Conv2D(out_ch, 1, activation='sigmoid')(o)
    return Model(inputs=i, outputs=o)

model = UNet(DIM, start_ch=32, dropout=0., batchnorm=True, residual=True)
model.compile(Adam(), dice_loss, [dice_coef])

model2 = UNet(DIM, start_ch=32, dropout=0., batchnorm=True, residual=True)
model2.compile(Adam(), dice_loss, [dice_coef])
#model2.summary()

In [ ]:
data_split = 10/10176
target_img_size = DIM[:2]
DATA = np.stack(([imgPath + i for i in os.listdir(imgPath)], [maskPath + i for i in os.listdir(maskPath)]), axis=1)
maxDataSplit = int(len(DATA) * data_split)
dataXArr = np.random.choice(len(DATA), maxDataSplit, replace=False)
xTrain = np.array([DATA[i][0] for i in dataXArr])
yTrain = np.array([DATA[i][1] for i in dataXArr])

def imgInPath(path):
    return image.img_to_array(image.load_img(path, target_size=target_img_size))
def maskInPath(path):
    return image.img_to_array(image.load_img(path, target_size=target_img_size, color_mode = "grayscale"))

x_batch = np.array([imgInPath(i) for i in xTrain])
y_batch = np.array([maskInPath(i) for i in yTrain])
x_batch = x_batch.astype('float32')
y_batch = y_batch.astype('float32')
x_batch /= 255
y_batch /= 255

In [ ]:
hystory = model.fit(x_batch, y_batch, batch_size=1, epochs=10, verbose=2, validation_split=0.)

In [ ]:
fit_on_batches(model=model2, x_data=x_batch, y_data=y_batch,
               epochs=10, batch_size=1, data_split=10176/10176, patience = None,
               mode='arr', target_img_size=DIM[:2])

In [ ]:
img = image.img_to_array(image.load_img('DATA/Stored/train/00087a6bd4dc_01.jpg', target_size=DIM[:2]))
mask = image.img_to_array(image.load_img('DATA/Stored/train_masks/00087a6bd4dc_01_mask.jpg', target_size=DIM[:2], color_mode = "grayscale"))
img = np.expand_dims(img, axis=0)
mask = np.expand_dims(mask, axis=0)
pred = model.predict(img)
pred2 = model2.predict(img)
plt.figure(figsize=(15,15))
plt.subplot(141)
plt.imshow(image.array_to_img(img[0]))
plt.subplot(142)
plt.imshow(image.array_to_img(mask[0]))
plt.subplot(143)
plt.imshow(image.array_to_img(pred[0]))
plt.subplot(144)
plt.imshow(image.array_to_img(pred2[0]))
plt.show()

In [ ]:
# Saving model
model_json = model.to_json()
json_file = open("Models/UNetModel.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("Models/UNetModel.h5")